In [35]:
class InternalPosition:
    def __init__(self, sender, counter, parent, left_child, depth, symbol):
        self.sender = sender
        self.counter = counter
        self.parent = parent
        self.left_child = left_child
        self.depth = depth
        self.symbol = symbol

    def set_counter(self, counter):
        self.counter = counter

    def set_parent(self, parent):
        self.parent = parent

    def __eq__(self, other):
        if not isinstance(other, InternalPosition):
            return False
        return (self.sender == other.sender and
                self.counter == other.counter and
                self.parent == other.parent and
                self.left_child == other.left_child and
                self.depth == other.depth and
                self.symbol == other.symbol)

    def __lt__(self, other):
        if not isinstance(other, InternalPosition):
            return NotImplemented
        return self.compare_to(other) < 0

    def __str__(self):
        return f"InternalPosition(sender={self.sender}, counter={self.counter}, parent={self.parent}, left_child={self.left_child}, depth={self.depth}, symbol={self.symbol})"

    def compare_to(self, other):
        if self == other:
            return 0

        a = self
        b = other

        last_move = None
        while a.depth > b.depth:
            last_move = ("a", a.left_child)
            a = a.parent
        while b.depth > a.depth:
            last_move = ("b", b.left_child)
            b = b.parent

        if a == b:
            return (1 if last_move[0] == "a" else -1) * (-1 if last_move[1] else 1)

        while a.parent != b.parent:
            a = a.parent
            b = b.parent

        if a.left_child and not b.left_child:
            return -1
        elif not a.left_child and b.left_child:
            return 1
        else:
            if a.sender < b.sender:
                return -1
            elif a.sender > b.sender:
                return 1
            else:
                return a.counter - b.counter


In [39]:
import bisect

class FugueTree:
    def __init__(self, replica_id):
        self.replica_id = replica_id
        self.local_state = []

    def create_between(self, a, b, symbol):
        is_ancestor = False
        if b is not None:
            if a is None:
                is_ancestor = True
            elif b.depth > a.depth:
                b_ancestor = b
                while b_ancestor.depth > a.depth:
                    b_ancestor = b_ancestor.parent
                if b_ancestor == a:
                    is_ancestor = True

        if is_ancestor:
            if a is None:
                new_pos = InternalPosition(self.replica_id, 0, None, True, 1, symbol)
            else:
                new_pos = InternalPosition(self.replica_id, 0, b, True, b.depth + 1, symbol)
        else:
            if a is None:
                new_pos = InternalPosition(self.replica_id, 0, None, False, 1, symbol)
            else:
                new_pos = InternalPosition(self.replica_id, 0, a, False, a.depth + 1, symbol)

        new_pos.set_counter(len(self.local_state))
        bisect.insort(self.local_state, new_pos)
        return new_pos

    def receive(self, internal_position):
        if internal_position.parent is not None and internal_position.parent not in self.local_state:
            raise ValueError("Wrong sending order, received child before parent")

        bisect.insort(self.local_state, internal_position)


In [40]:
import logging
from collections import deque
from typing import Optional

class CrdtService:
    def __init__(self, replica_id: str):
        self.fugue_tree = FugueTree(replica_id)
        self.positions_to_send = deque()
        self.replica_id = replica_id
        self.peers = []

    def add_peer(self, peer):
        self.peers.append(peer)

    def receive(self, internal_position: InternalPosition):
        logging.info(f"Received new position: {internal_position}")
        self.fugue_tree.receive(internal_position)
        self.log_current_state()

    def insert(self, pos: int, symb: str):
        if pos < 0:
            raise RuntimeError("Position must not be negative!")
        
        if pos >= len(self.fugue_tree.local_state):
            logging.info(f"Inserting {symb} as last symbol")
            inserted = self.add_last(symb) if self.fugue_tree.local_state else self.insert_between(None, None, symb)
        elif pos == 0:
            logging.info(f"Inserting {symb} as first symbol")
            inserted = self.add_first(symb)
        else:
            array_view = list(self.fugue_tree.local_state)
            logging.info(f"Inserting {symb} at position: {pos}, between {array_view[pos - 1].symbol} and {array_view[pos].symbol}")
            inserted = self.insert_between(array_view[pos - 1], array_view[pos], symb)

        self.log_current_state()
        self.positions_to_send.append(inserted)

    def get_local_text(self) -> str:
        positions = list(self.fugue_tree.local_state)
        return ''.join(pos.symbol for pos in positions)

    def send_changes_to_peers(self):
        if self.positions_to_send:
            logging.info(f"Found {len(self.positions_to_send)} not positions to send")
            while self.positions_to_send:
                pos = self.positions_to_send.popleft()
                for peer in self.peers:
                    peer.receive(pos)

    def add_first(self, symb: str) -> InternalPosition:
        return self.fugue_tree.create_between(None, next(iter(self.fugue_tree.local_state), None), symb)

    def add_last(self, symb: str) -> InternalPosition:
        return self.fugue_tree.create_between(next(reversed(self.fugue_tree.local_state), None), None, symb)

    def insert_between(self, a: Optional[InternalPosition], b: Optional[InternalPosition], symb: str) -> InternalPosition:
        return self.fugue_tree.create_between(a, b, symb)

    def log_current_state(self):
        logging.info(f"Text on replica {self.replica_id}: {self.get_local_text()}")


In [41]:
import logging

logging.basicConfig(level=logging.INFO)

editor1 = CrdtService(replica_id='replica1')
editor2 = CrdtService(replica_id='replica2')

editor1.add_peer(editor2)
editor2.add_peer(editor1)

editor1.insert(0, 'H')
editor1.send_changes_to_peers()
editor1.insert(1, 'e')
editor1.insert(2, 'l')
editor1.insert(3, 'l')
editor1.insert(4, 'o')
editor1.insert(5, ' ')

editor2.insert(1, 'W')
editor2.send_changes_to_peers()
editor2.insert(2, 'o')
editor2.insert(3, 'r')
editor2.insert(4, 'l')
editor2.insert(5, 'd')

editor1.send_changes_to_peers()
editor2.send_changes_to_peers()

editor2.insert(11,'!')

print("Editor 1 text:", editor1.get_local_text())
print("Editor 2 text:", editor2.get_local_text())

INFO:root:Inserting H as last symbol
INFO:root:Text on replica replica1: H
INFO:root:Found 1 not sent positions, sending...
INFO:root:Received new position: InternalPosition(sender=replica1, counter=0, parent=None, left_child=False, depth=1, symbol=H)
INFO:root:Text on replica replica2: H
INFO:root:Inserting e as last symbol
INFO:root:Text on replica replica1: He
INFO:root:Inserting l as last symbol
INFO:root:Text on replica replica1: Hel
INFO:root:Inserting l as last symbol
INFO:root:Text on replica replica1: Hell
INFO:root:Inserting o as last symbol
INFO:root:Text on replica replica1: Hello
INFO:root:Inserting   as last symbol
INFO:root:Text on replica replica1: Hello 
INFO:root:Inserting W as last symbol
INFO:root:Text on replica replica2: HW
INFO:root:Found 1 not sent positions, sending...
INFO:root:Received new position: InternalPosition(sender=replica2, counter=1, parent=InternalPosition(sender=replica1, counter=0, parent=None, left_child=False, depth=1, symbol=H), left_child=Fal

Editor 1 text: Hello World
Editor 2 text: Hello World!
